In [ ]:
from pyspark import SparkContext
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.catalog import read, read_value, query, write
from geopyspark.geotrellis.constants import SPATIAL, ZOOM, TILE
from geopyspark.geotrellis.geotiff_rdd import get
from geopyspark.geotrellis.rdd import RasterRDD, TiledRasterRDD
from geonotebook.vis.geotrellis.render_methods import single_band_render_from_color_map
from geonotebook.wrappers import GeoTrellisCatalogLayerData, RddRasterData
import numpy as np

In [ ]:
catalog_uri = "file://catalog"

In [ ]:
# Gather GeoTiffs

!curl -o /tmp/DevelopedLand.tiff https://s3.amazonaws.com/chattademo/DevelopedLand.tiff
!curl -o /tmp/FarmlandOrForestedLandsWithPrimeAgriculturalSoils.tiff https://s3.amazonaws.com/chattademo/FarmlandOrForestedLandsWithPrimeAgriculturalSoils.tiff
!curl -o /tmp/FarmlandWithoutPrimeAgriculturalSoils.tiff.tiff https://s3.amazonaws.com/chattademo/FarmlandWithoutPrimeAgriculturalSoils.tiff.tiff
!curl -o /tmp/ForestedLands.tiff https://s3.amazonaws.com/chattademo/ForestedLands.tiff
!curl -o /tmp/ImperviousSurfacesBarrenLandsOpenWater.tiff https://s3.amazonaws.com/chattademo/ImperviousSurfacesBarrenLandsOpenWater.tiff
!curl -o /tmp/NonWorkingProtectedOrPublicLands.tiff https://s3.amazonaws.com/chattademo/NonWorkingProtectedOrPublicLands.tiff
!curl -o /tmp/PrimeAgriculturalSoilsNotForestedOrFarmland.tiff https://s3.amazonaws.com/chattademo/PrimeAgriculturalSoilsNotForestedOrFarmland.tiff
!curl -o /tmp/PrivatelyOwnedWorkingLandsWithEasements.tiff https://s3.amazonaws.com/chattademo/PrivatelyOwnedWorkingLandsWithEasements.tiff
!curl -o /tmp/PublicallyOwnedWorkingLands.tiff https://s3.amazonaws.com/chattademo/PublicallyOwnedWorkingLands.tiff
!curl -o /tmp/Wetlands.tiff https://s3.amazonaws.com/chattademo/Wetlands.tiff


In [ ]:
# Get the mask

!curl -o /tmp/mask.json https://s3.amazonaws.com/chattademo/chatta_mask.json

In [ ]:
from functools import partial
import fiona
import json
import pyproj
from shapely.geometry import mapping, shape
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

txt = open('/tmp/mask.json').read()
js = json.loads(txt)
geom = shape(js)
center = geom.centroid
chatta_poly = transform(project, geom)
chatta_poly

In [ ]:
layers = { "developed-land" : "file:///tmp/DevelopedLand.tiff",
           "prime-ag-soil" : "file:///tmp/FarmlandOrForestedLandsWithPrimeAgriculturalSoils.tiff",
           "not-prime-ag-soil" : "file:///tmp/FarmlandOrForestedLandsWithPrimeAgriculturalSoils.tiff",
           "forested-lands" : "file:///tmp/ForestedLands.tiff",
           "impervious" : "file:///tmp/ImperviousSurfacesBarrenLandsOpenWater.tiff",
           "protected" : "file:///tmp/NonWorkingProtectedOrPublicLands.tiff",
           "prime-ag-soil-notfarm" : "file:///tmp/PrimeAgriculturalSoilsNotForestedOrFarmland.tiff",
           "private-with-easements" : "file:///tmp/PrivatelyOwnedWorkingLandsWithEasements.tiff",
           "public-working-lands" : "file:///tmp/PublicallyOwnedWorkingLands.tiff",
           "wetlands" : "file:///tmp/Wetlands.tiff" }

In [ ]:
sc = SparkContext(appName="Chatta Ingest")
geopysc = GeoPyContext(sc)

In [ ]:
def ingest_layer(layer_name, path, mask=True):
    rdd = get(geopysc, SPATIAL, path, maxTileSize=512, numPartitions=20)
    metadata = rdd.collect_metadata(tile_size=512)
    metadata['cellType'] = "int8"
    laid_out = rdd.tile_to_layout(metadata)
    reprojected = laid_out.reproject("EPSG:3857", scheme=ZOOM)
    if mask:
        target = reprojected.mask([chatta_poly])
    else:
        target = reprojected
    for layer_rdd in target.pyramid(reprojected.zoom_level, 0):
        write(catalog_uri, layer_name, layer_rdd)    

In [ ]:
for layer in layers:
    print("Ingesting %s at %s" % (layer, layers[layer]))
    ingest_layer(layer, layers[layer])


In [ ]:
# debug
ingest_layer("developed-land-nomask", layers['developed-land'], mask=False)

In [ ]:
M.set_center(-85.2935099666654, 35.20529964318362, 9)

In [ ]:
cmap = { 0 : "#000000FF", 1: "#FF9D57FF", -128: "#00000000" }
cmap_render = single_band_render_from_color_map(cmap)
def render_tile(tile):
    arr = tile[0]
    return cmap_render(arr)
data = GeoTrellisCatalogLayerData(geopysc, 
                                  catalog_uri, 
                                  "developed-land",
                                  SPATIAL)
M.add_layer(data, render_tile=render_tile)

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
# debug

v = read_value(geopysc, SPATIAL, catalog_uri, "developed-land-nomask", 11, 540, 807)
t = v['data']
render_tile(t)

In [ ]:
M.layers